In [0]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [0]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#reading dataset harem for relation extraction
import pandas as pd
dh = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/dataset.ptbr_harem.csv',delimiter=';',sep='delimiter', header=None, encoding="UTF-8",
                   names=['SENTENCE_ID','RELATION_ID','SENTENCE','ARGUMENT_1','ARGUMENT_1_CATEGORY','RELATION','ARGUMENT_2','ARGUMENT_2_CATEGORY']) 
#Display first 10 rows
dh.head(10)

,SENTENCE_ID,RELATION_ID,SENTENCE,ARGUMENT_1,ARGUMENT_1_CATEGORY,RELATION,ARGUMENT_2,ARGUMENT_2_CATEGORY
0,SENTENCE_ID,RELATION_ID,SENTENCE,ARGUMENT_1,ARGUMENT_1_CATEGORY,RELATION,ARGUMENT_2,ARGUMENT_2_CATEGORY
1,3,1,"Marcelo_Tejedor, paleontólogo de a Universidad...",Marcelo_Tejedor,PER,NaN,Universidade_Nacional_da_Patagônia,ORG
2,4,3,Esse é um alvo viável para remédios contra a o...,John_Clapham,PER,NaN,Smith_Kline_Beecham,ORG
3,9,2,Um grupo que reúne 13 sociedades científicas n...,Associação_Nacional_de_Biossegurança,ORG,NaN,ANBio,ORG
4,10,1,"Primeiro, pede que a CTNBio (Comissão_Técnica_...",CTNBio,ORG,NaN,Comissão_Técnica_Nacional_de_Biossegurança,ORG
5,11,1,A idéia foi lançada por o ministro_Roberto_Ama...,ministro_Roberto_Amaral,PER,NaN,Ciência_e_Tecnologia,ORG
6,11,5,A idéia foi lançada por o ministro_Roberto_Ama...,USP,ORG,NaN,Universidade_de_São_Paulo,ORG
7,12,1,"Francelino_Grando, secretário_de_Política_Tecn...",Francelino_Grando,PER,NaN,MCT,ORG
8,12,2,"Francelino_Grando, secretário_de_Política_Tecn...",secretário_de_Política_Tecnológica_Empresarial,PER,NaN,MCT,ORG
9,12,3,"Francelino_Grando, secretário_de_Política_Tecn...",MCT,ORG,NaN,Ministério_da_Ciência_e_Tecnologia,ORG


In [0]:
dh = dh.fillna(method = 'ffill')

In [0]:
# This is a class to get sentence.
class sentence(object):
    def __init__(self, dh):
        self.n_sent = 1
        self.dh = dh
        self.empty = False
        agg = lambda s : [(s, a1,rel ) for s, a1, rel 
                                                   in zip(s['SENTENCE'].values.tolist(),
                                                       s['ARGUMENT_1'].values.tolist(),
                                                       s['ARGUMENT_1_CATEGORY'].values.tolist())]
        self.grouped = self.dh.groupby("RELATION_ID").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['RELATION_ID: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
#Displaying one full sentence
getter = sentence(dh)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

"Marcelo_Tejedor, paleontólogo de a Universidade_Nacional_da_Patagônia, esboça um sorriso meio sem graça quando lhe perguntam se não é uma situação frustrante. Primeiro, pede que a CTNBio (Comissão_Técnica_Nacional_de_Biossegurança) seja a única instância a regulamentar tanto a pesquisa como a comercialização de transgênicos, em o âmbito de o mérito científico. A idéia foi lançada por o ministro_Roberto_Amaral (Ciência_e_Tecnologia) e detalhada ontem durante a abertura de o 1º Congresso_Internacional_de_Biodiesel, realizado em Ribeirão_Preto e promovido por a USP (Universidade_de_São_Paulo) de a cidade. Francelino_Grando, secretário_de_Política_Tecnológica_Empresarial de o MCT (Ministério_da_Ciência_e_Tecnologia), disse que a proposta é 'uma equação lógica'. Os brasileiros (Gilson_Rambelli, Paulo_Bava_de_Camargo e Flávio_Rizzi, pesquisadores de o Museu_de_Arqueologia_e_Etnologia (MAE) de a USP) acharam mesmo restos de um ou dois navios antigos, possivelmente portugueses de os século 15

In [0]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [0]:
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

sent = preprocess(sentences[0])
print(sent)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[('Marcelo_Tejedor', 'NNP'), (',', ','), ('paleontólogo', 'NN'), ('de', 'IN'), ('a', 'DT'), ('Universidade_Nacional_da_Patagônia', 'NNP'), (',', ','), ('esboça', 'VBZ'), ('um', 'JJ'), ('sorriso', 'NN'), ('meio', 'NN'), ('sem', 'NN'), ('graça', 'NN'), ('quando', 'NN'), ('lhe', 'NN'), ('perguntam', 'NN'), ('se', 'NN'), ('não', 'JJ'), ('é', 'NNP'), ('uma', 'NN'), ('situação', 'NN'), ('frustrante', 'NN'), ('.', '.'), ('Primeiro', 'NNP'), (',', ','), ('pede', 'NN'), ('que', 'NN'), ('a', 'DT'), ('CTNBio', 'NNP'), ('(', '('), ('Comissão_Técnica_Nacional_de_Biossegurança', 'NNP'), (')', ')'), ('seja', 'VBD'), ('a', 'DT'), ('única', 'NNP'), ('instância', 'NN'), ('a', 'DT'), ('regulamentar', 'NN'), ('t

In [0]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  Marcelo_Tejedor/NNP
  ,/,
  (NP paleontólogo/NN)
  de/IN
  a/DT
  Universidade_Nacional_da_Patagônia/NNP
  ,/,
  esboça/VBZ
  (NP um/JJ sorriso/NN)
  (NP meio/NN)
  (NP sem/NN)
  (NP graça/NN)
  (NP quando/NN)
  (NP lhe/NN)
  (NP perguntam/NN)
  (NP se/NN)
  não/JJ
  é/NNP
  (NP uma/NN)
  (NP situação/NN)
  (NP frustrante/NN)
  ./.
  Primeiro/NNP
  ,/,
  (NP pede/NN)
  (NP que/NN)
  a/DT
  CTNBio/NNP
  (/(
  Comissão_Técnica_Nacional_de_Biossegurança/NNP
  )/)
  seja/VBD
  a/DT
  única/NNP
  (NP instância/NN)
  (NP a/DT regulamentar/NN)
  tanto/IN
  (NP a/DT pesquisa/NN)
  (NP como/NN)
  (NP a/DT comercialização/NN)
  de/IN
  (NP transgênicos/NN)
  ,/,
  (NP em/JJ o/NN)
  âmbito/NNP
  de/IN
  o/FW
  mérito/FW
  (NP científico/NN)
  ./.
  (NP A/DT idéia/JJ foi/NN)
  (NP lançada/NN)
  (NP por/NN)
  o/JJ
  ministro_Roberto_Amaral/JJ
  (/(
  Ciência_e_Tecnologia/NNP
  )/)
  e/VBP
  (NP detalhada/JJ ontem/NN)
  durante/VBZ
  (NP a/DT abertura/NN)
  de/IN
  o/FW
  1º/CD
  Congresso_Inte

In [0]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
iob_tagged = tree2conlltags(cs)
pprint(iob_tagged)

[('Marcelo_Tejedor', 'NNP', 'O'),
 (',', ',', 'O'),
 ('paleontólogo', 'NN', 'B-NP'),
 ('de', 'IN', 'O'),
 ('a', 'DT', 'O'),
 ('Universidade_Nacional_da_Patagônia', 'NNP', 'O'),
 (',', ',', 'O'),
 ('esboça', 'VBZ', 'O'),
 ('um', 'JJ', 'B-NP'),
 ('sorriso', 'NN', 'I-NP'),
 ('meio', 'NN', 'B-NP'),
 ('sem', 'NN', 'B-NP'),
 ('graça', 'NN', 'B-NP'),
 ('quando', 'NN', 'B-NP'),
 ('lhe', 'NN', 'B-NP'),
 ('perguntam', 'NN', 'B-NP'),
 ('se', 'NN', 'B-NP'),
 ('não', 'JJ', 'O'),
 ('é', 'NNP', 'O'),
 ('uma', 'NN', 'B-NP'),
 ('situação', 'NN', 'B-NP'),
 ('frustrante', 'NN', 'B-NP'),
 ('.', '.', 'O'),
 ('Primeiro', 'NNP', 'O'),
 (',', ',', 'O'),
 ('pede', 'NN', 'B-NP'),
 ('que', 'NN', 'B-NP'),
 ('a', 'DT', 'O'),
 ('CTNBio', 'NNP', 'O'),
 ('(', '(', 'O'),
 ('Comissão_Técnica_Nacional_de_Biossegurança', 'NNP', 'O'),
 (')', ')', 'O'),
 ('seja', 'VBD', 'O'),
 ('a', 'DT', 'O'),
 ('única', 'NNP', 'O'),
 ('instância', 'NN', 'B-NP'),
 ('a', 'DT', 'B-NP'),
 ('regulamentar', 'NN', 'I-NP'),
 ('tanto', 'IN', 'O')

In [0]:
import nltk

nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk import ne_chunk, pos_tag, word_tokenize 
ne_tree = ne_chunk(pos_tag(word_tokenize(sentences[0])))
print(ne_tree)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
(S
  (GPE Marcelo_Tejedor/NNP)
  ,/,
  paleontólogo/NN
  de/IN
  a/DT
  (ORGANIZATION Universidade_Nacional_da_Patagônia/NNP)
  ,/,
  esboça/VBZ
  um/JJ
  sorriso/NN
  meio/NN
  sem/NN
  graça/NN
  quando/NN
  lhe/NN
  perguntam/NN
  se/NN
  não/JJ
  é/NNP
  uma/NN
  situação/NN
  frustrante/NN
  ./.
  Primeiro/NNP
  ,/,
  pede/NN
  que/NN
  a/DT
  (ORGANIZATION CTNBio/NNP)
  (/(
  (ORGANIZATION Comissão_Técnica_Nacional_de_Biossegurança/NNP)
  )/)
  seja/VBD
  a/DT
  única/NNP
  instância/NN
  a/DT
  regulamentar/NN
  tanto/IN
  a/DT
  pesquisa/NN
  como/NN
  a/DT
  comercialização/NN
  de/IN
  transgênicos/NN
  ,/,
  em/JJ
  o/NN
  âmbito/NNP
  de/IN
  o/FW
  mérito/FW
  científico/NN
  ./.
  A/DT
  idéia/JJ
  foi/NN
  lançada

In [0]:
import nltk
import re

pos_tags = nltk.pos_tag(word_tokenize(sentences[0])) 
ne = nltk.ne_chunk(pos_tags)  

chunked = nltk.ne_chunk_sents(pos_tags, binary=True)

IN = re.compile(r'.*\bde\b(?!\b.+ndo)')
IN2 = re.compile(r'.*\bpor\b(?!\b.+ndo)')
IN3 = re.compile(r'.*\bem\b(?!\b.+ndo)')

for rel in nltk.sem.extract_rels('PERSON','ORGANIZATION', ne, corpus='ace', pattern=IN):
    print(nltk.sem.rtuple(rel))

for rel in nltk.sem.extract_rels('ORGANIZATION','ORGANIZATION', ne, corpus='ace', pattern=IN):
    print(nltk.sem.rtuple(rel))       

for rel2 in nltk.sem.extract_rels('GPE','ORGANIZATION', ne, corpus='ace', pattern=IN):
    print(nltk.sem.rtuple(rel2))       

for rel in nltk.sem.extract_rels('ORGANIZATION','PERSON', ne, corpus='ace', pattern=IN2):
    print(nltk.sem.rtuple(rel))

for rel in nltk.sem.extract_rels('PERSON','ORGANIZATION', ne, corpus='ace', pattern=IN2):
    print(nltk.sem.rtuple(rel))

for rel in nltk.sem.extract_rels('PERSON','ORGANIZATION', ne, corpus='ace', pattern=IN3):
    print(nltk.sem.rtuple(rel))          
  
for rel in nltk.sem.extract_rels('ORGANIZATION','ORGANIZATION', ne, corpus='ace', pattern=IN3):
    print(nltk.sem.rtuple(rel))   

for rel in nltk.sem.extract_rels('PERSON','GPE', ne, corpus='ace', pattern=IN3):
    print(nltk.sem.rtuple(rel))   

[PER: 'Flávio_Rizzi/NNP'] ',/, pesquisadores/VBZ de/FW o/FW' [ORG: 'Museu_de_Arqueologia_e_Etnologia/NNP']
[PER: 'Para/NNP Greg_Laden/NNP'] ',/, de/IN a/DT' [ORG: 'Universidade_de_Minnesota/NNP']
[PER: 'Richard_Wrangham/NNP'] ',/, de/IN a/DT' [ORG: 'Universidade_Harvard/NNP']
[PER: 'Mark_Mattson/NNP'] ',/, de/FW o/FW Instituto_Nacional_do_Envelhecimentos/NNP de/FW os/FW' [ORG: 'EUA/NNP']
[PER: 'Reunião/NNP Anual/NNP'] 'de/IN a/DT' [ORG: 'AAAS/NNP']
[PER: 'Benjamin_Wolozin/NNP'] ',/, de/FW o/FW' [ORG: 'Centro_Médico_da_Universidade_Loyola/NNP']
[PER: 'Bioantropólogo/NNP'] 'de/IN a/DT' [ORG: 'Universidade_MacMaster/NNP']
[PER: 'Douglas_Revelle/NNP'] ',/, de/FW o/FW' [ORG: 'Laboratório_Nacional_de_Los_Alamos/NNP']
[PER: 'Robert_Jedicke/NNP'] ',/, de/IN a/DT' [ORG: 'Universidade_do_Arizona/NNP']
[PER: 'Fernando_Gomes/NNP'] ',/, presidente/NN de/FW a/DT' [ORG: 'Câmara_Municipal_do_Porto/NNP']
[PER: 'António_Fontes/NNP'] ',/, de/IN a/DT' [ORG: 'AIPAN/NNP']
[PER: 'Habitualmente/NNP'] 'usado/N